# Data Preparation

This notebook outlines the essential data preparation steps required for subsequent data analysis and model training. The dataset consists of 38 individual datasets, each representing one day of machine data. All datasets are concatenated into a single dataset, which is then shuffled and split into two subsets: 80% for training and 20% for testing.

Later, the training subset is further split into training and validation sets using an 80/20 ratio. This results in 64% of the overall data being used for model training, 16% for validation, and 20% for testing.

The data preparation process includes the following key steps:
1. **Data Cleaning:** Removing unnecessary columns, handling missing or invalid values, and converting numeric columns to the correct data types.
2. **Preserving Timestamps:** The `_time` column is kept in the concatenated dataset for future analysis but is removed from the training and testing datasets to avoid interference during model training.
3. **Label Encoding:** Converting machine state labels into numerical values for compatibility with machine learning models.
4. **Feature Scaling:** Scaling all numeric features (excluding labels) using `StandardScaler` to ensure features are centered around zero with a standard deviation of one. The scaler is fitted on the training data and applied to the test data to avoid data leakage.

This pipeline ensures that the data is cleaned, standardized, and prepared in a consistent manner for machine learning tasks, while maintaining the integrity of the test dataset for unbiased model evaluation.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import re
from datetime import datetime
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import joblib

In [2]:
# Load and concatenate datasets

print("Loading data...")
folder_path = 'labeled_data'
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
dataframes = [pd.read_csv(file, sep=";", low_memory=False) for file in all_files]
concatenated_df = pd.concat(dataframes, ignore_index=True)
print(f"Loaded {len(dataframes)} files with {concatenated_df.shape[0]} rows in total.")

Loading data...
Loaded 38 files with 2883832 rows in total.


In [3]:
# Data Cleaning

print("Cleaning data...")
concatenated_df = concatenated_df.drop(columns=['Unnamed: 0', 'value'], errors='ignore')
concatenated_df = concatenated_df.loc[concatenated_df['Label'] != '0']
concatenated_df = concatenated_df.replace(',', '.', regex=True)
columns_to_convert = [col for col in concatenated_df.columns if col not in ['Label', '_time']]
concatenated_df[columns_to_convert] = concatenated_df[columns_to_convert].astype(float)
if '_time' in concatenated_df.columns:
    concatenated_df['_time'] = pd.to_datetime(concatenated_df['_time'])
print(f"Cleaned data has {concatenated_df.shape[0]} rows and {concatenated_df.shape[1]} columns.")

Cleaning data...
Cleaned data has 1346830 rows and 44 columns.


In [4]:
# Check for duplicates (ignoring the '_time' column)

print("Checking for duplicates...")
columns_to_check = [col for col in concatenated_df.columns if col != '_time']
duplicate_rows = concatenated_df.duplicated(subset=columns_to_check, keep=False)
num_duplicates = duplicate_rows.sum()

if num_duplicates > 0:
    print(f"Warning: {num_duplicates} duplicate rows found (ignoring the '_time' column).")
else:
    print("No duplicate rows found (ignoring the '_time' column).")

# Remove duplicates ignoring the '_time' column
concatenated_df = concatenated_df.drop_duplicates(subset=columns_to_check)

# Output the new number of rows and columns
print(f"After removing duplicates, data has {concatenated_df.shape[0]} rows and {concatenated_df.shape[1]} columns.")

Checking for duplicates...
After removing duplicates, data has 968717 rows and 44 columns.


In [5]:
# Label Encoding

print("Encoding labels...")
unique_labels = concatenated_df['Label'].unique()
replace_dict = {label: idx for idx, label in enumerate(unique_labels)}
concatenated_df['Label'] = concatenated_df['Label'].replace(replace_dict)
print("Label encoding completed. Mapping:")
for label, idx in replace_dict.items():
    print(f"  {label}: {idx}")

Encoding labels...
Label encoding completed. Mapping:
  Block einladen: 0
  Seitenbesäumung: 1
  Kleben: 2
  Schopfbesäumung: 3
  Produktion: 4
  Bodenhaut entfernen: 5
  Stillstand: 6


In [6]:
# Split the dataset

print("Splitting data into training and test sets...")

random_indices = np.random.RandomState(seed=42).permutation(len(concatenated_df.index))
shuffled_df = concatenated_df.iloc[random_indices]
split_index = int(len(shuffled_df) * 0.8)

train_df = shuffled_df.iloc[:split_index].copy()
test_df = shuffled_df.iloc[split_index:].copy()
print(f"Training set: {len(train_df)} rows\nTest set: {len(test_df)} rows")

# Remove '_time' from training and test datasets
if '_time' in train_df.columns:
    train_df = train_df.drop(columns=['_time'])
if '_time' in test_df.columns:
    test_df = test_df.drop(columns=['_time'])

Splitting data into training and test sets...
Training set: 774973 rows
Test set: 193744 rows


In [7]:
# Verification

# Verify dataset splitting
total_rows = len(concatenated_df)
split_rows = len(train_df) + len(test_df)
if total_rows == split_rows:
    print("Dataset splitting is consistent.")
else:
    print(f"Mismatch in dataset splitting: Total={total_rows}, Train+Test={split_rows}")

# Check for duplicate rows between training and test datasets
train_hashes = pd.util.hash_pandas_object(train_df, index=False)
test_hashes = pd.util.hash_pandas_object(test_df, index=False)

# Prüfen auf gemeinsame Hashes
common_hashes = set(train_hashes).intersection(set(test_hashes))
if common_hashes:
    print(f"Warning: {len(common_hashes)} common rows found between training and test datasets.")
else:
    print("No common rows between training and test datasets.")
    
# Verify label consistency between training and test datasets
missing_labels = set(test_df['Label']) - set(train_df['Label'])
if missing_labels:
    print(f"Warning: Test dataset contains labels not present in training: {missing_labels}")
else:
    print("Label encoding is consistent between training and test datasets.")

Dataset splitting is consistent.
No common rows between training and test datasets.
Label encoding is consistent between training and test datasets.


In [8]:
from sklearn.preprocessing import normalize

# Scale Features
print("Normalizing data...")
label_column = 'Label'

train_scaled = train_df.copy()
test_scaled = test_df.copy()

train_features = train_df.loc[:, train_df.columns != label_column]
test_features = test_df.loc[:, test_df.columns != label_column]

train_scaled.loc[:, train_df.columns != label_column] = normalize(train_features, axis=0, norm='l2')
test_scaled.loc[:, test_df.columns != label_column] = normalize(test_features, axis=0, norm='l2')

print("Normalization completed.")

Normalizing data...
Normalization completed.


In [9]:
# Berechne die L2-Norm für jede Zeile
norm_values = (train_scaled.loc[:, train_df.columns != label_column] ** 2).sum(axis=0)

# Überprüfen, ob jede Zeile die L2-Norm von 1 hat
print(norm_values)

# Berechne die L2-Norm für jede Zeile
norm_values2 = (test_scaled.loc[:, test_df.columns != label_column] ** 2).sum(axis=0)

# Überprüfen, ob jede Zeile die L2-Norm von 1 hat
print(norm_values2)

AggHoeheIst                   1.0
AutomTurmverstellungEin       0.0
AutomatikLaeuft               1.0
BSR_Satznummer                1.0
BSR_Schnittstaerke            1.0
BSR_StueckzahlIst             1.0
BSR_StueckzahlSoll            1.0
BSVE_LaengeIst                1.0
BSVE_LaengeSoll               1.0
BSVE_Satznummer               1.0
BSVE_Schnittstaerke           1.0
BSVE_StueckzahlIst            1.0
BSVE_StueckzahlSoll           1.0
BetriebsartBSR                0.0
BetriebsartBSVE               1.0
BetriebsartHalbautomat        0.0
BetriebsartManuell            1.0
BetriebsartService            0.0
Blocklaenge120m               1.0
DrwHoeheIst                   1.0
DrwHoeheOffsetAutomatik       1.0
HTBVIst                       1.0
HTB_OffsetSoll                1.0
HTB_StromIst                  1.0
HTB_TemperaturIst             1.0
HTB_VIst                      1.0
Halbautomat_Satznummer        1.0
Halbautomat_Schnittstaerke    1.0
Halbautomat_StueckzahlIst     1.0
Halbautomat_St

In [10]:
train_scaled.head()

,AggHoeheIst,AutomTurmverstellungEin,AutomatikLaeuft,BSR_Satznummer,BSR_Schnittstaerke,BSR_StueckzahlIst,BSR_StueckzahlSoll,BSVE_LaengeIst,BSVE_LaengeSoll,BSVE_Satznummer,...,Reserve04,SAOAbstandIst,SAUAbstandIst,VDrwIst,VMesserIst,VMesserSoll,VWicklerIst,WinkelIst,WinkelSoll,Label
923244,0.000450,0.0,0.001322,0.001136,0.000386,0.000263,0.001336,0.000858,0.000668,0.001136,...,0.0,0.0,0.000713,0.0,0.001271,0.00114,0.000693,0.001136,0.0,4
1315202,0.002093,0.0,0.000000,0.001136,0.001087,0.000008,0.000007,0.000000,0.000542,0.001136,...,0.0,0.0,0.001074,0.0,0.000000,0.00114,0.000000,0.001136,0.0,1
924347,0.000412,0.0,0.001322,0.001136,0.000386,0.000269,0.001336,0.000455,0.000668,0.001136,...,0.0,0.0,0.000713,0.0,0.001271,0.00114,0.000705,0.001136,0.0,4
2538883,0.002093,0.0,0.000000,0.001136,0.001138,0.002621,0.001336,0.000000,0.000271,0.001136,...,0.0,0.0,0.001462,0.0,0.000000,0.00114,0.000422,0.001136,0.0,0
2667296,0.000703,0.0,0.001322,0.001136,0.000726,0.002737,0.001336,0.000629,0.000777,0.001136,...,0.0,0.0,0.001031,0.0,0.001271,0.00114,0.001532,0.001136,0.0,4


In [11]:
test_scaled.head()

,AggHoeheIst,AutomTurmverstellungEin,AutomatikLaeuft,BSR_Satznummer,BSR_Schnittstaerke,BSR_StueckzahlIst,BSR_StueckzahlSoll,BSVE_LaengeIst,BSVE_LaengeSoll,BSVE_Satznummer,...,Reserve04,SAOAbstandIst,SAUAbstandIst,VDrwIst,VMesserIst,VMesserSoll,VWicklerIst,WinkelIst,WinkelSoll,Label
1250624,0.001613,0.0,0.000000,0.002272,0.001602,0.000312,0.001223,0.000000,0.004044,0.002272,...,0.0,0.0,0.002232,0.0,0.002539,0.002279,0.002230,0.002272,0.0,3
2669702,0.001085,0.0,0.002643,0.002272,0.001450,0.005529,0.002674,0.000481,0.001553,0.002272,...,0.0,0.0,0.002062,0.0,0.002539,0.002279,0.002895,0.002272,0.0,4
1671076,0.000374,0.0,0.002643,0.002272,0.002271,0.000040,0.000021,0.001071,0.001083,0.002272,...,0.0,0.0,0.002186,0.0,0.002539,0.002279,0.003229,0.002272,0.0,4
526507,0.002363,0.0,0.002643,0.002272,0.001465,0.000556,0.001750,0.000417,0.000289,0.002272,...,0.0,0.0,0.002341,0.0,0.002539,0.002279,0.003111,0.002272,0.0,3
1939325,0.001861,0.0,0.002643,0.002272,0.002320,0.001633,0.002674,0.000527,0.000722,0.002272,...,0.0,0.0,0.002274,0.0,0.002539,0.002279,0.003451,0.002272,0.0,4


In [12]:
# Apply SMOTE

print("Applying SMOTE to training data...")
smote = SMOTE(random_state=42)
train_features = train_scaled.drop(columns=label_column)
train_labels = train_scaled[label_column]

train_features_oversampled, train_labels_oversampled = smote.fit_resample(train_features, train_labels)
train_oversampled = pd.DataFrame(train_features_oversampled, columns=train_features.columns)
train_oversampled[label_column] = train_labels_oversampled.values

# Check class distribution
unique, counts = np.unique(train_labels_oversampled, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Class distribution after SMOTE:")
for cls, count in class_counts.items():
    print(f"  Class {cls}: {count} samples")

Applying SMOTE to training data...
Class distribution after SMOTE:
  Class 0: 502130 samples
  Class 1: 502130 samples
  Class 2: 502130 samples
  Class 3: 502130 samples
  Class 4: 502130 samples
  Class 5: 502130 samples
  Class 6: 502130 samples


## Data Export

The processed datasets are exported as CSV files to ensure they can be reused for further analysis or model training without needing to repeat the data preparation steps.

In [13]:
# Save processed datasets

print("Saving processed datasets...")

concatenated_df.to_csv('processed_datasets_new/data_with_timestamps_cleaned.csv', index=False)
train_scaled.to_csv('processed_datasets_new/training_data_scaled.csv', index=False)
test_scaled.to_csv('processed_datasets_new/testing_data_scaled.csv', index=False)
train_oversampled.to_csv('processed_datasets_new/training_data_scaled_smote.csv', index=False)
print("All datasets saved successfully.")

Saving processed datasets...
All datasets saved successfully.
